In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from faker import Faker
from datetime import datetime

pd.set_option("display.max_colwidth", 5000)

In [2]:
df_dic_influd21 = pd.read_excel("dicionario_sindrome_respiratoria.xlsx")
dic_columns_raw = pd.Series( list( df_dic_influd21.DBF ) ).dropna().to_numpy()

# Organiza os nomes das colunas separando em novos nomes aqueles campos separados por " OU "
dic_columns = []
for col in dic_columns_raw:
    col = col.upper()
    if(" OU " in col):
        col = col.split(" OU ")
        dic_columns.append(col[0])
        dic_columns.append(col[1])
    else: 
        dic_columns.append(col.replace(" OU", ""))
dic_columns = np.array(dic_columns)

In [3]:
df_influd21 = pd.read_csv("../../INFLUD21-09-06-2022.csv", on_bad_lines='skip', sep = ";")
df_influd21.head(2)

/tmp/ipykernel_29382/3883492426.py:1: DtypeWarning: Columns (15,59,61,62,63,91,93,105,107,114,116,117,118,120,122,143) have mixed types. Specify dtype option on import or set low_memory=False.
  df_influd21 = pd.read_csv("../../INFLUD21-09-06-2022.csv", on_bad_lines='skip', sep = ";")


,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,DOSE_2_COV,DOSE_REF,FAB_COV_1,FAB_COV_2,FAB_COVREF,LOTE_REF,LAB_PR_COV,LOTE_1_COV,LOTE_2_COV,FNT_IN_COV
0,05/01/2021,1,04/01/2021,1,AL,1 MICRORREGIAO DE SAUDE,1533.0,MACEIO,270430,HOSPITAL DA MULHER DRA NISE DA SILVEIRA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,06/01/2021,1,03/01/2021,1,MS,EX ARS DE CAMPO GRANDE,1975.0,CAMPO GRANDE,500270,PRONCOR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [4]:
def search_field(df, field, return_index = False):
    '''
    Encontra colunas que contém em parte o texto presente em field
    '''
    
    if(not return_index):
        return df.loc[:,np.array([field in column for column in df.columns])]
    return np.array([field in column for column in df.columns])

In [5]:
np.mean(pd.Series(list(search_field(df_influd21, "CS_ESCOL_N").iloc[:,0])).isna())

0.35355039767623586

In [6]:
# verifica quais colunas da base fornecida de fato pertencem ao dicionário dos dados oferecido
df_columns_in_dic = np.array([col in dic_columns for col in df_influd21.columns])
dic_in_df_columns = np.array([col in df_influd21.columns for col in dic_columns])

In [7]:
# Tem na base e não tem no dicionário. Estranho...
tem_base_nao_tem_dic = df_influd21.columns[~df_columns_in_dic]

# Tem no dicionário e não tem na base. Provavelmente confidenciais...
tem_dic_nao_tem_base = dic_columns[~dic_in_df_columns] 

In [8]:
# Colunas (com descrição) dos campos presentes no dicionário da base de dados que não estão presentes na base atual obtida
# no portal do SUS para Síndrome Respiratória Aguda
df_dic_influd21.loc[df_dic_influd21.DBF.isin(tem_dic_nao_tem_base),["DBF","Descrição","Características"]]

,DBF,Descrição,Características
0,NU_NOTIFIC,Número do registro,"Campo Interno\nNúmero sequencial gerado automaticamente pelo sistema. Utilizar o padrão:\n320120000123\nDígito 1: caracteriza o tipo da ficha (1=SG, 2=SRAG-UTI e 3-SRAG Hospitalizado).\nDígitos 2 a 12: número sequencial gerado automaticamente pelo sistema."
11,TEM_CPF,Informar se o paciente notificado dispõe de Número do Cadastro de Pessoa Física (CPF),"Campo Obrigatório\nSe selecionado “Sim”, preencher campo “CPF”. Se selecionado “Não” preencher CNS. Se o paciente não dispor de CPF é obrigatório o preenchimento do CNS. No caso de pacientes raça/cor indígenas, somente o CNS é considerado como campo obrigatório."
12,NU_CPF,Número do Cadastro de Pessoa Física (CPF) do paciente notificado,"Campo Obrigatório\nQuando preenchido o número do CPF o sistema deverá preencher o Nome, Sexo, Data de Nascimento, Idade, Raça/Cor e o nome da mãe do paciente."
14,NU_CNS,Preencher com o número do Cartão Nacional de Saúde do paciente,Campo Obrigatório
15,NM_PACIENT,Nome completo do,Campo Obrigatório
24,CS_ETINIA,"Nome e código da etnia do paciente, quando indígena.",Campo Essencial\nHabilitado se campo\n11-Raça/Cor for igual a 5-Indígena.
25,POV_CT,Informar se o paciente for membro de algum povo ou comunidade tradicional,Campo Obrigatório
26,TP_POV_CT,Informar o povo ou comunidade tradicional,Campo Obrigatório- Habilitado se campo\n13- É membro de povo ou comunidade tradicional? for igual a 1- Sim
30,NM_MAE_PAC,Nome completo da mãe do paciente (sem abreviações).,Campo Essencial
31,NU_CEP,CEP de residência do paciente.,Campo Essencial\nValidado a partir da tabela de CEP dos Correios.


Veja que esses campos provavelmente foram deletados de modo a remover informações confidenciais dos pacientes do Sistema Único de Saúde. Como temos o interesse em desenvolver uma plataforma que usa esses dados confidenciais para fazer o link de bases de dados, a partir deste momento trabalharemos com uma base de dados mockada para o desenvolvimento do modelo de ligação entre indivíduos.

A priori utilizarmos os campos

### Variáveis relacionadas a informações pessoais do paciente
* TEM_CPF: Se o paciente tem ou não CPF (1: Sim, 2: Não);
* NU_CPF: Caso o paciente tenha CPF, seu número do CPF;
* NU_CNS: Número do Cartão Nacional de Sáude;
* NM_PACIENT: Nome completo do paciente;
* NM_MAE_PAC: Nome completo da mãe do paciente;
* DT_NASC: Data de nascimento do paciente;
* CS_SEXO: Sexo do paciente;
* CS_RACA: Raça do paciente;

### Variáveis relacionadas ao endereço do paciente
* SG_UF_NOT: Estado (Unidade Federativa) do paciente;
* ID_MUNICIP: Município atual do paciente;
* NM_LOGRADO: Logradouro (rua, avenida, quadra, ...)
* NU_NUMERO: Número do logradouro (casa ou edifício);
* NM_BAIRRO: Bairro de residência do paciente;
* NM_COMPLEM: Complemento do logradouro (casa, apto, bloco, ...);

Esses campos foram baseados na observação das colunas faltantes da tabela e na checagem de campos presentes no formulário de notificação do ESUS para casos suspeitos de Covid. O treinamento do modelo será baseado na linkagem da base de casos suspeitos do ESUS notifica com a base do SRAG 2021. Os links para ambos os formulários são dados a seguir:

Formulário ESUS notifica: https://sinepe-pe.org.br/wp-content/uploads/2020/10/ficha_e_sus.pdf

Formulário SRAG: https://www.saude.go.gov.br/files/vigilancia/epidemiologica/fichas-de-notificacao/SindromeRespiratoriaAgudaGrave-SIVEPGRIPE.pdf.pdf

Base de dados de SRAG: https://opendatasus.saude.gov.br/dataset/srag-2021-e-2022/resource/dd91a114-47a6-4f21-bcd5-86737d4fc734#

É interessante destacarmos que, embora a base de dados de SRAG 2021 tenha muito mais informações sobre os pacientes, existe um gargalo das informações que podem ser utilizadas na classificação dos pacientes, já que para a efetuação do linkage é necessário que ambas as bases compartilhem os campos usados para a comparação.

### Análise de valores reais

Nessa seção vejamos o conteúdo das colunas selecionadas para comparação que, por ventura, também estejam presentes na base de dados de SRAG 2021.

In [9]:
# Variáveis usadas para o linkage das bases
link_vars = ["TEM_CPF","NU_CPF","NU_CNS","NM_PACIENT","NM_MAE_PAC",
             "DT_NASC","CS_SEXO","CS_ETINIA","NU_CEP","SG_UF_NOT",
             "ID_MUNICIP","NM_LOGRADO","NU_NUMERO","NM_BAIRRO","NM_COMPLEM"]

print("Número de variáveis para o link das bases de dados:", len(link_vars))

valid_link_vars = []
for link_var in link_vars:
    if(link_var in df_influd21.columns):
        valid_link_vars.append( link_var )
df = df_influd21.loc[:,valid_link_vars]

print("Dimensão da tabela com os dados presentes:", df.shape)
df.head(2)

Embora temos apenas 4 desses campos presentes na base de dados, podemos verificar valores como o número de observações presentes na base e a proporção de observações faltantes (para termos uma noção do ruído acrescentado na base de treinamento do modelo)

Temos observações faltantes apenas na coluna DT_NASC

In [10]:
print("Número de observações com data faltante:", len(np.where(df.DT_NASC.isna())[0]))
print("Proporção total: ", np.round(len(np.where(df.DT_NASC.isna())[0]) / df.shape[0] * 100, 4), "%", sep="")

Número de observações com data faltante: 1188
Proporção total: 0.0687%


### Mock database

Na seção seguinte faremos a geração de valores _dummy_ (_fake_) para as informações confidenciais a serem linkadas. 

In [11]:
from mock_data import *

In [12]:
def create_patient_database(n_patients = 1000):
    columns = ['TEM_CPF', 'NU_CPF', 'TEM_CNS', 'NU_CNS', 'NM_PACIENT',
               'NM_MAE_PAC', 'DT_NASC', 'CS_SEXO', 'CS_RACA', 'SG_UF_NOT',
               'ID_MUNICIP', 'NM_LOGRADO', 'NU_NUMERO', 'NM_BAIRRO', 'NM_COMPLEM']
    rows = []
    for i in range(n_patients):
        patient = create_patient()
        rows.append(list(patient.values()))
    df = pd.DataFrame(rows)
    df.columns = columns
    return df

In [13]:
df = create_patient_database(n_patients = 1000)
df.shape

(1000, 15)

In [15]:
df.head(15)

,TEM_CPF,NU_CPF,TEM_CNS,NU_CNS,NM_PACIENT,NM_MAE_PAC,DT_NASC,CS_SEXO,CS_RACA,SG_UF_NOT,ID_MUNICIP,NM_LOGRADO,NU_NUMERO,NM_BAIRRO,NM_COMPLEM
0,False,None,True,517164270685320,Fernando Anunciacao de Deus,Maria Anunciacao de Deus Elias,1974-02-17,1,1,CE,Bela Cruz,Estrada de Azevedo,453,Conjunto Celso Machado,
1,True,265.037.419-52,True,991590920949183,Aparecida de Magalhaes,Terezinha de Magalhaes,1928-01-26,2,1,PI,Redenção do Gurgueia,Feira de Silveira,450,Santa Monica,
2,True,723.085.469-56,False,None,João Claudino de Paiva,Maria Claudino de Paiva,1976-10-14,1,1,RJ,Macuco,Rua Correia,28,Corumbiara,APT 124
3,True,469.350.712-16,False,None,Roberto Tenorio Evangelista,Sueli Tenorio Evangelista,1998-02-11,9,1,RS,Gramado Xavier,Campo Isabel Duarte,6,Ouro Preto,
4,True,421.965.870-01,False,None,Antonio Miguel Dionizio,Aparecida Miguel Dionizio,1941-01-21,9,4,PB,Juripiranga,Colônia Raul Santos,736,Bernadete,
5,True,064.275.398-92,True,001766843298640,Vanderlei Venceslau Hipolito,Maria Venceslau Hipolito,1980-11-28,1,1,GO,Iporá,Avenida Moreira,349,Pindorama,APT 124
6,True,526.390.748-65,True,256211796202672,Sonia Dutra Rabelo Sodre,Maria Dutra Rabelo Sodre,1971-03-08,9,9,PR,Cornélio Procópio,Praia Rocha,81,Flavio Marques Lisboa,APT 49
7,True,973.182.450-23,True,313830347068231,Maria da Luz do Rego,Eliane da Luz do Rego,1968-02-19,2,9,PR,São Mateus do Sul,Área Novaes,65,Nova Suíça,APT 122
8,True,976.385.012-68,True,132492702664340,Julio Schmidt de Resende,Rosa Schmidt de Resende Limeira,1999-05-01,9,4,PA,Rurópolis,Praça de Teixeira,998,Unidas,APT 88
9,True,347.529.681-00,False,None,Afonso Goulart,Maria Goulart,1961-04-20,1,4,MG,Minduri,Vale Mendes,61,Baleia,
